(c) This notebook is an asset of: https://github.com/tubml-pathology/xMIL-Pathways

Please note the license and citation instructions as described in the above repository.

In [ ]:
import scipy.stats as stats
import numpy as np
import random
import json

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from statsmodels.stats.multitest import multipletests
import os

### Global variables

In [ ]:
base_path_project = Path("/path/to/zenodo_sample/results")  # SPECIFY THIS *********************
base_data_path = base_path_project / "ihc_he_analysis"

storing_path = base_path_project / 'boxplots'

SAVE = True
if SAVE:
    storing_path.mkdir(parents=True, exist_ok=True)    

### Load the data

In [ ]:
base_path = storing_path
pathway_name_mapping= {'pstat3':'JAK-STAT/pSTAT3'}

dfs = {}
for pathway in ['cd34', 'tnfa', 'pstat3']:
    fn = base_path / f"{pathway}.csv"
    try:
        df = pd.read_csv(fn, index_col=0)
        df['Pathway'] = pathway_name_mapping[pathway]
        dfs[pathway] = df
    except FileNotFoundError as e:
        print(f"Could not find {fn=}. Continuing .. ")


In [ ]:
dfs[next(iter(dfs))].head()

In [ ]:
df = pd.concat(dfs.values()).reset_index(drop=True)

In [ ]:
df

### Define the columns

In [ ]:
X_AXIS_COL = 'case_id'
Y_AXIS_COL = 'sum_cell_act_cont'
ROW_COL = 'Pathway'
COL_COL = 'cls_smooth_with_border'
HUE_COL = 'mean_lrp_sign'

In [ ]:
case_id_name_mapping = {'P6': 'P6'}

### Compute the distribution significance

Compute significant distribution for each sample, pathway and tissue compartment. Correct for multiple testing within each pathway.

In [ ]:
alpha = 0.01

grouped_df = df.groupby([
    X_AXIS_COL, 
    ROW_COL,
    COL_COL,
])

# for each sample pathway and tissue type compute the distribution 
sign_res = []
for group_name, group_data in grouped_df:
    group_neg = group_data[group_data['mean_lrp_sign'] == -1][Y_AXIS_COL].dropna()
    group_pos = group_data[group_data['mean_lrp_sign'] == 1][Y_AXIS_COL].dropna()

    statistic, p_value = stats.mannwhitneyu(group_neg, group_pos, alternative='less')

    sign_res.append(
        (*group_name, case_id_name_mapping.get(group_name[0], group_name[0]), statistic, p_value)
    )
sign_res = pd.DataFrame(sign_res, columns=[X_AXIS_COL, ROW_COL, COL_COL, "case_name_mapping", "statistic_MWU", "pval_MWU"])

# mutliple testing correction within each pathway 
for group_name, group_data in sign_res.groupby(ROW_COL):
    fdr_rejected, fdr_pvals, _, _ = multipletests(group_data['pval_MWU'], alpha=alpha, method='fdr_bh')
    sign_res.loc[group_data.index, "pval_adj_MWU"] = fdr_pvals
    sign_res.loc[group_data.index, "fdr_rejected_MWU"] = fdr_rejected

sign_res = sign_res.set_index([X_AXIS_COL, ROW_COL, COL_COL])
sign_res.head()

if SAVE:
    sign_res.to_csv(storing_path / "mean_lrp_distr_test_results_per_sample_pathway_tissue_region.csv")
    print(f"Saved mean_lrp_distr_test_results_per_sample_pathway_tissue_region at {storing_path / 'mean_lrp_distr_test_results_per_sample_pathway_tissue_region.csv'}")

Compute significant distribution for each sample and pathway (**aggregated across tissue compartments**). Correct for multiple testing within each pathway.

In [ ]:
alpha = 0.01

grouped_df = df.groupby([
    X_AXIS_COL,
    ROW_COL,
])

# for each sample pathway and tissue type compute the distribution 
sign_res_aggr = []
for group_name, group_data in grouped_df:
    group_neg = group_data[group_data['mean_lrp_sign'] == -1][Y_AXIS_COL].dropna()
    group_pos = group_data[group_data['mean_lrp_sign'] == 1][Y_AXIS_COL].dropna()

    statistic, p_value = stats.mannwhitneyu(group_neg, group_pos, alternative='less')

    sign_res_aggr.append(
        (*group_name, case_id_name_mapping.get(group_name[0], group_name[0]), statistic, p_value)
    )
sign_res_aggr = pd.DataFrame(sign_res_aggr, columns=[X_AXIS_COL, ROW_COL, "case_name_mapping", "statistic_MWU", "pval_MWU"])

# mutliple testing correction within each pathway 
for group_name, group_data in sign_res_aggr.groupby(ROW_COL):
    fdr_rejected, fdr_pvals, _, _ = multipletests(group_data['pval_MWU'], alpha=alpha, method='fdr_bh')
    sign_res_aggr.loc[group_data.index, "pval_adj_MWU"] = fdr_pvals
    sign_res_aggr.loc[group_data.index, "fdr_rejected_MWU"] = fdr_rejected

sign_res_aggr = sign_res_aggr.set_index([X_AXIS_COL, ROW_COL])
sign_res_aggr.head()

if SAVE:
    sign_res_aggr.to_csv(storing_path / "mean_lrp_distr_test_results_per_sample_pathway.csv")
    print(f"Saved mean_lrp_distr_test_results_per_sample_pathway at {storing_path / 'mean_lrp_distr_test_results_per_sample_pathway.csv'}")


### Plot the box plot

In [ ]:
plt.rcParams.update({'font.size': 12})  # General font size
plt.rcParams.update({
    'axes.titlesize': 16,      # Subplot titles
    'axes.labelsize': 14,      # Axis labels
    'xtick.labelsize': 12,     # X-axis tick labels
    'ytick.labelsize': 12,     # Y-axis tick labels
    'legend.fontsize': 14,     # Legend
    'figure.titlesize': 16     # Main title
})
order_pathways = ['JAK-STAT/pSTAT3', 'VEGF/CD34']
order_comps = ['non-tumor' , 'border', 'tumor']
color_map = ['#80b1d3', '#fb8072']

In [ ]:

# for showfliers in [True, False]:
for showfliers in [False]:
    ### MAIN PLOT 
    g = sns.catplot(
        data=df.sort_values(X_AXIS_COL), 
        x=X_AXIS_COL, 
        y=Y_AXIS_COL, 
        hue=HUE_COL,
        col=COL_COL,
        col_order=order_comps,
        kind="box", 
        height=3.75,
        aspect=1.6,
        palette=color_map,
        sharey=False,
        showfliers=showfliers,
        fliersize = 1
    )
        
    g.set_titles("")
    g.set_xlabels("")
    g.set_ylabels("Sum Cell Activity (cont.)")

    for i, comp in enumerate(order_comps):
        g.axes[0, i].set_title(comp.capitalize())

    
    ### Modify box colors based on the significance in sign_res object 
    
    def modify_box_colors_based_on_significance(data, **kwargs):
        import matplotlib.patches as mpatches
        ax = plt.gca()
    
        # get current tissue region and pathway combi
        row = data[ROW_COL].unique()
        col = data[COL_COL].unique()
        if len(row)!=1 or len(col)!=1:
            raise ValueError()
    
        row = row[0]
        col = col[0]
        
        all_patches = ax.patches
        box_patches = [p for p in all_patches if isinstance(p, mpatches.PathPatch)]
        cases = sorted(data[X_AXIS_COL].unique())
        cases4boxes = cases*2
        if len(box_patches) != len(cases4boxes):
            raise ValueError(f"{len(box_patches)=} not equal to {len(cases4boxes)=}")
    
        for patch, case in zip(box_patches, cases4boxes):
            is_significant = sign_res.loc[(case, row, col), 'fdr_rejected_MWU']
    
            face_color = patch.get_facecolor()
            if is_significant:
                patch.set_alpha(0.8)
            else:
                patch.set_alpha(0.2)
                
            patch.set_edgecolor(face_color)
            patch.set_linewidth(0.75)

        ## map xticklabels
        if ax.get_xticklabels():
            ax.set_xticklabels([
                case_id_name_mapping.get(text.get_text(), text.get_text()) 
                for text in ax.get_xticklabels()
            ])
    
    
    g.map_dataframe(modify_box_colors_based_on_significance)
    
    
    ### Legend things
    sns.move_legend(g, "lower center", bbox_to_anchor=(0.5, -0.25), ncol=len(df[HUE_COL].unique()), title="Sign mean LRP values",
                   title_fontsize=14)
    
    ## Final spacing
    plt.tight_layout()
    g.fig.subplots_adjust(wspace=0.1)
    
    if SAVE:
        fn = f"distribution_cell_activations_{ 'with_fliers' if showfliers else 'wo_fliers'}"
        plt.savefig(storing_path / f'{fn}.pdf', bbox_inches='tight')
        plt.savefig(storing_path / f'{fn}.svg', bbox_inches='tight')
        plt.savefig(storing_path / f'{fn}.png', dpi=600, bbox_inches='tight')
        print(f"Stored figure at {storing_path / f'{fn}.pdf'}")
    
    plt.show(g.fig)

### Aggregated across compartments

In [ ]:

# for showfliers in [True, False]:
for showfliers in [False]:
    ### MAIN PLOT 
    g = sns.catplot(
        data=df.sort_values(X_AXIS_COL), 
        x=X_AXIS_COL, 
        y=Y_AXIS_COL, 
        hue=HUE_COL,
        
        kind="box", 
        height=3.75,
        aspect=1.6,
        palette=color_map,
        sharey=False,
        showfliers=showfliers,
        fliersize = 1
    )
        
    g.set_titles("")
    g.set_xlabels("")
    g.set_ylabels("Sum Cell Activity (cont.)")

    
    ### Modify box colors based on the significance in sign_res_aggr object 
    def modify_box_colors_based_on_significance(data, **kwargs):
        import matplotlib.patches as mpatches
        ax = plt.gca()
    
        row = data[ROW_COL].unique()
        if len(row)!=1:
            raise ValueError()
    
        row = row[0]
        
        all_patches = ax.patches
        box_patches = [p for p in all_patches if isinstance(p, mpatches.PathPatch)]
        cases = sorted(data[X_AXIS_COL].unique())
        cases4boxes = cases*2
        if len(box_patches) != len(cases4boxes):
            raise ValueError(f"{len(box_patches)=} not equal to {len(cases4boxes)=}")
    
        for patch, case in zip(box_patches, cases4boxes):
            is_significant = sign_res_aggr.loc[(case, row), 'fdr_rejected_MWU']
    
            face_color = patch.get_facecolor()
            if is_significant:
                patch.set_alpha(0.8)
            else:
                patch.set_alpha(0.2)
                
            patch.set_edgecolor(face_color)
            patch.set_linewidth(0.75)

        ## map xticklabels
        if ax.get_xticklabels():
            ax.set_xticklabels([
                case_id_name_mapping.get(text.get_text(), text.get_text()) 
                for text in ax.get_xticklabels()
            ])
    
    
    g.map_dataframe(modify_box_colors_based_on_significance)
    
    
    ### Legend things
    sns.move_legend(g, "lower center", bbox_to_anchor=(0.5, -0.25), ncol=len(df[HUE_COL].unique()), title="Sign mean LRP values",
                   title_fontsize=14)
    
    ## Final spacing
    plt.tight_layout()
    g.fig.subplots_adjust(hspace=0.2)
    
    if SAVE:
        fn = f"distribution_cell_activations_aggregated_across_compartments_{ 'with_fliers' if showfliers else 'wo_fliers'}"
        plt.savefig(storing_path / f'{fn}.pdf', bbox_inches='tight')
        plt.savefig(storing_path / f'{fn}.svg', bbox_inches='tight')
        plt.savefig(storing_path / f'{fn}.png', dpi=600, bbox_inches='tight')
        print(f"Stored figure at {storing_path / f'{fn}.pdf'}")
    
    plt.show(g.fig)